In [9]:
import time
import glob as glob
from zipfile import ZipFile
from bs4 import BeautifulSoup
import dateutil.parser as dparser
from datetime import datetime
import pandas as pd
import multiprocessing as mp
import numpy as np

In [10]:
df = pd.read_csv('/home/shane/Documents/thesis/output/final/failures.csv')
df.head()

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,source,service,timestamp_dir,timestamp_site,success_count,success_pct,partial_count,partial_pct,major_count,major_pct,maintenance_count,maintenance_pct,unknown_count,unknown_pct
0,slack,slack,20200508T150001,NaN,10,1.0,0,0.0,0,0.0,0,0.0,0,0.0
1,slack,slack,20200508T060001,NaN,10,1.0,0,0.0,0,0.0,0,0.0,0,0.0
2,slack,slack,20200508T130001,NaN,10,1.0,0,0.0,0,0.0,0,0.0,0,0.0
3,slack,slack,20200508T200001,NaN,10,1.0,0,0.0,0,0.0,0,0.0,0,0.0
4,slack,slack,20200508T030001,NaN,9,0.9,1,0.1,0,0.0,0,0.0,0,0.0


### Mean % of statuses per source and service

In [27]:
df_agg = df.groupby(['source','service']).agg(mean_success=('success_pct',np.mean), mean_partial=('partial_pct',np.mean), mean_major=('major_pct',np.mean), mean_maintenance=('maintenance_pct',np.mean), mean_unknown=('unknown_pct',np.mean))
df_agg

mean_success  mean_partial  mean_major  \
source        service                                              
atlassian     access          1.000000      0.000000    0.000000   
              bitbucket       0.994156      0.001622    0.000817   
              confluence      0.994570      0.004568    0.000318   
              developers      0.998662      0.000660    0.000429   
              jira-align      1.000000      0.000000    0.000000   
...                                ...           ...         ...   
downrightnow  typepad         1.000000      0.000000    0.000000   
              yahoomail       0.995968      0.004032    0.000000   
              youtube         1.000000      0.000000    0.000000   
github-status github          0.996964      0.002711    0.000325   
slack         slack           0.980163      0.019837    0.000000   

                          mean_maintenance  mean_unknown  
source        service                                     
atlassian     access              0.000000      0.000000  
              bitbucket           0.001702      0.001702  
              confluence          0.000272      0.000272  
              developers          0.000124      0.000124  
              jira-align          0.000000      0.000000  
...                                    ...           ...  
downrightnow  typepad             0.000000      0.000000  
              yahoomail           0.000000      0.000000  
              youtube             0.000000      0.000000  
github-status github              0.000000      0.000000  
slack         slack               0.000000      0.000000  

[188 rows x 5 columns]